In [ ]:
import os

import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from timeit import default_timer as timer

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score
from statistics import mean
from sklearn.metrics import hamming_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import learning_curve

from sklearn.metrics import roc_auc_score, confusion_matrix
import statistics
from sklearn.metrics import recall_score

from wordcloud import WordCloud
from collections import Counter

from sklearn.pipeline import Pipeline

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
os.chdir("./drive/My Drive/question recommendation dataset")

In [63]:
!ls

Questions.csv


In [117]:
import pandas as pd

dataset = pd.read_csv("./Questions.csv")

In [ ]:
dataset.shape

(95, 4)

In [119]:
dataset.head()

,Unnamed: 0,Questions,Subject,Tags,Body
0,0,What is Artificial Intelligence?,AI,['AI'],"I am just starting, can you tell me what is ar..."
1,1,What is machine learning?,AI,"['ML', 'Supervised Learning', 'Unsupervised Le...","Please help me in this, i am just starting thi..."
2,2,What is deep learning?,AI,"['DL', 'Neural Networks', 'Backpropagation']","I really want to learn this, what to do about ..."
3,3,What is reinforcement learning?,AI,"['RL', 'Markov Decision Processes', 'Q-Learning']","This is really confusing, how can i solve it. ..."
4,4,What is supervised learning?,AI,"['ML', 'Supervised Learning', 'Classification'...",My exam in near. I really want to know what is...


In [66]:


def tokenize(text):
    '''
    Tokenize text and return a non-unique list of tokenized words found in the text. 
    Normalize to lowercase, strip punctuation, remove stop words, filter non-ascii characters.
    Lemmatize the words and lastly drop words of length < 3.
    '''
    text = text.lower()
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text)
    words = nopunct.split(' ')
    # remove any non ascii
    words = [word.encode('ascii', 'ignore').decode('ascii') for word in words]
    lmtzr = WordNetLemmatizer()
    words = [lmtzr.lemmatize(w) for w in words]
    words = [w for w in words if len(w) > 2]
    return words

In [67]:
vector = TfidfVectorizer(ngram_range=(1, 1), analyzer='word',
                         tokenizer=tokenize, stop_words='english',
                         strip_accents='unicode', use_idf=1, min_df=10)

In [118]:
dataset['Body'] = 10
for i in range(dataset.shape[0]):
  if i%10==0:
    dataset['Body'].loc[i] = 'I am just starting, can you tell me ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 1:
    dataset['Body'].loc[i] = 'Please help me in this, i am just starting this, can you please tell me ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 2:
    dataset['Body'].loc[i] = 'I really want to learn this, what to do about this? the question is : ' + dataset['Questions'].loc[i].lower() 
  elif i%10 == 3:
    dataset['Body'].loc[i] = 'This is really confusing, how can i solve it. The question is:  ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 4:
    dataset['Body'].loc[i] = 'My exam in near. I really want to know ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 5:
    dataset['Body'].loc[i] = 'Anyone experts here, please help me in this. ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 6:
    dataset['Body'].loc[i] = 'I have been trying to find this questions answer for a long time. ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 7:
    dataset['Body'].loc[i] = 'Hello everyone. this question is really troubling me for a long time, the question is ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 8:
    dataset['Body'].loc[i] = 'namaste guys. can you give me short answer on ' + dataset['Questions'].loc[i].lower()
  elif i%10 == 9:
    dataset['Body'].loc[i] = 'This question when i asked on ChatGPT gives too complex answer. can you give me simple answer, the question is ' + dataset['Questions'].loc[i].lower()
  


In [82]:
for ind,val in dataset['Subject'].items():
  dataset['Subject'].loc[ind] = val.lower()

for ind,val in dataset['Tags'].items():
  dataset['Tags'].loc[ind] = val.lower()

In [ ]:
import joblib

joblib.dump(dataset,'Questions.csv')

['Questions.csv']

In [ ]:
import nltk
from nltk.stem.porter import *

In [84]:
dataset = dataset.iloc[:,0:-1]

In [110]:
dataset.tail(25)

,Unnamed: 0,Questions,Subject,Tags,query
70,70,How can managers create a positive organizatio...,management,['organizational culture'],How can managers create a positive organizatio...
71,71,What is the role of emotional intelligence in ...,management,['emotional intelligence'],What is the role of emotional intelligence in ...
72,72,Why is talent management important?,management,['talent management'],Why is talent management important?management[...
73,73,What is the purpose of HTML?,webdevelopment,['html'],What is the purpose of HTML?webdevelopment['ht...
74,74,What is the purpose of CSS?,webdevelopment,"['css', 'purpose', 'styling', 'selectors']",What is the purpose of CSS?webdevelopment['css...
75,75,What is the difference between inline and bloc...,webdevelopment,"['html', 'elements', 'inline', 'block']",What is the difference between inline and bloc...
76,76,What is the difference between padding and mar...,webdevelopment,"['css', 'box model', 'padding', 'margin']",What is the difference between padding and mar...
77,77,What is the box model in CSS?,webdevelopment,"['css', 'box model', 'elements', 'padding']",What is the box model in CSS?webdevelopment['c...
78,78,What is a CSS selector?,webdevelopment,"['css', 'selector', 'styling', 'class']","What is a CSS selector?webdevelopment['css', '..."
79,79,What is a CSS framework?,webdevelopment,"['css', 'framework', 'bootstrap', 'responsive ...","What is a CSS framework?webdevelopment['css', ..."


In [85]:
dataset['query'] = dataset['Questions'] + dataset['Subject'] + dataset['Tags']

In [86]:
query = vector.fit_transform(dataset['query']).toarray()

In [87]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(query)


In [88]:
a = sorted(list(enumerate(similarity[0])) , reverse=True ,key=lambda x:x[1])
a[0:10]

[(0, 0.0),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0)]

In [114]:
def recommend_using_sub(subject):
    sub = subject.lower()
    id = dataset[dataset['Subject']==sub].index[0]
    
#     print(movie_id)
    calculate = sorted(list(enumerate(similarity[id])) , reverse=True , key=lambda x:x[1])
#     print(calculate)
    for i in calculate[0:10]:
#         print(i)
        print(dataset.iloc[i[0]].Questions)

In [112]:
def recommend_using_id(id):
#     print(movie_id)
    calculate = sorted(list(enumerate(similarity[id])) , reverse=True , key=lambda x:x[1])
#     print(calculate)
    for i in calculate[0:10]:
#         print(i)
        print(dataset.iloc[i[0]].Questions)


In [115]:
print(recommend_using_sub('ai'))
print(recommend_using_id(10))

What is Artificial Intelligence?
What is machine learning?
What is deep learning?
What is reinforcement learning?
What is supervised learning?
What is unsupervised learning?
What is semi-supervised learning?
What is transfer learning?
What is a neural network?
What are the types of neural networks?
None
What is Artificial Intelligence?
What is machine learning?
What is deep learning?
What is reinforcement learning?
What is supervised learning?
What is unsupervised learning?
What is semi-supervised learning?
What is transfer learning?
What is a neural network?
What are the types of neural networks?
None


In [116]:
import joblib
joblib.dump(similarity, 'similarity.pkl')

['similarity.pkl']

In [120]:
dataset.to_csv("Questions_with_answers.csv")